Source of LSTM codes:
https://github.com/quantumiracle/Popular-RL-Algorithms/blob/master/sac_v2_lstm.py

In [ ]:

import gymnasium as gym
import numpy as np
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from sb3_contrib import RecurrentPPO
from sb3_contrib.ppo_recurrent import MlpLstmPolicy, CnnLstmPolicy, MultiInputLstmPolicy


# Import packages

In [ ]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break


%load_ext autoreload
%autoreload 2

import sys
from data_wrangling import specific_utils, process_monkey_information, base_processing_class
from pattern_discovery import pattern_by_trials, pattern_by_points, make_ff_dataframe, ff_dataframe_utils, pattern_by_trials, pattern_by_points, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils, plot_cluster_replacement
from decision_making_analysis.decision_making import decision_making_utils, plot_decision_making, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_helper_class, GUAT_collect_info_class, GUAT_combine_info_class, add_features_GUAT_and_TAFT
from decision_making_analysis import free_selection, replacement, trajectory_info
from visualization.matplotlib_tools import plot_trials, plot_polar, additional_plots, plot_behaviors_utils, plot_statistics
from visualization.animation import animation_func, animation_utils, animation_class
from null_behaviors import sample_null_distributions, show_null_trajectory
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from reinforcement_learning.base_classes import env_utils, base_env, more_envs, rl_base_class, rl_base_utils
from reinforcement_learning.agents.rnn import gru_utils, lstm_utils, lstm_utils, lstm_class
from reinforcement_learning.agents.rppo import rppo_class
from reinforcement_learning.agents.feedforward import interpret_neural_network, sb3_class, sb3_utils
from eye_position_analysis import eye_positions
from neural_data_analysis.neural_analysis_tools.model_neural_data import neural_data_modeling
from decision_making_analysis.compare_GUAT_and_TAFT import find_GUAT_or_TAFT_trials
from reinforcement_learning.agents.feedforward import interpret_neural_network, sb3_class, sb3_utils

import os, sys
import torch
import numpy as np
import pickle
from gymnasium import spaces, Env
import torch
import optuna
from numpy import pi
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from torch.linalg import vector_norm
from IPython.display import HTML
from functools import partial
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
import gc
from importlib import reload
plt.rcParams["animation.html"] = "html5"
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

device_idx = 0
# device = torch.device("cuda:" + str(device_idx) if torch.cuda.is_available() else "cpu")
## if using Jupyter Notebook
device = "mps" if torch.backends.mps.is_available() else "cpu"
PLAYER = "agent"

In [ ]:
stop!

In [ ]:
!pwd

In [ ]:
!which python

# RPPO

Iterate through agents stored in 'RL_models/RPPO_stored_models/obs4_mem3_rank_keep_160627' that starts with 'dv', 

In [ ]:
all_agents_folder = 'multiff_analysis/RL_models/RPPO_stored_models/all_agents'
# list the directories in all_agents_folder
all_agents = os.listdir(all_agents_folder)
all_agents

In [ ]:
#overall_folder = 'multiff_analysis/RL_models/RPPO_stored_models/all_agents/lr3e-4_numobs3_mem2_stratdrop_fill_nenvs1_job160109_6/'
#overall_folder = 'multiff_analysis/RL_models/RPPO_stored_models/all_agents/example2/'
#overall_folder ="multiff_analysis/RL_models/RPPO_stored_models/obs4_mem3_rank_keep_160627/dv0p2_dw5p0_w2p0"
overall_folder ="multiff_analysis/RL_models/RPPO_stored_models/quick_test"
env_kwargs = {'num_obs_ff': 1,
              'add_action_to_obs': True,
              'angular_terminal_vel': 0.05,
              "dt": 0.1,
              "max_in_memory_time": 2,
            }   
rl = rppo_class.RPPOforMultifirefly(overall_folder=overall_folder,
                                                **env_kwargs)


rl.streamline_everything(currentTrial_for_animation=None, num_trials_for_animation=None, duration=[10, 40],
                         to_load_latest_agent=True,
                         best_model_postcurriculum_exists_ok=True,
                         to_train_agent=True,
                         use_curriculum_training=True
                         )
                                             

## see animation

In [ ]:
os.path.basename(os.path.normpath(overall_folder))


In [ ]:
rl.env_for_data_collection.dv_cost_factor

In [ ]:
from pathlib import Path
import sys
from datetime import datetime


In [ ]:
base_path = "multiff_analysis/RL_models/RPPO_stored_models/all_agents/ff5_mem3_drop_fill_lr3e-4_nenvs1_job160811_16"
base = Path(base_path).expanduser()

# If this run has fine-tunes under ft/, list those as agents
target = base / "ft" if (base / "ft").is_dir() else base
print(f"[info] Listing agents under: {target}")

# Collect immediate child directories
dirs = [p for p in target.iterdir() if p.is_dir()]
dirs.sort(key=lambda p: p.name)

In [ ]:
str(agent_dir)

## just to collect data

In [ ]:


import matplotlib.pyplot as plt
import seaborn as sns

col = 'num_stops'

cost_results2 = cost_results[cost_results['agent_name'].str.contains('175252', na=False)]

df_mean = cost_results2.groupby(['dv', 'jerk', 'stop'], as_index=False)[col].mean()

 
unique_dv = sorted(df_mean['dv'].unique())

for dv_value in unique_dv:
    df_plot = df_mean[df_mean['dv'] == dv_value].pivot(index='stop', columns='jerk', values=col)
    plt.figure(figsize=(6, 4))
    sns.heatmap(df_plot, annot=True, cmap='magma', fmt='.0f')
    plt.title(f'dv = {dv_value}')
    plt.xlabel('jerk')
    plt.ylabel('stop')
    plt.show()


In [ ]:
import pandas as pd
cost_results = pd.DataFrame()
for agent_dir in dirs:
    agent_name = os.path.basename(agent_dir)
    print('==============================================')
    print(agent_name)
    
    if '175252' not in agent_name:
        continue
    
    rl = rppo_class.RPPOforMultifirefly(overall_folder=str(agent_dir))

    rl.collect_data(n_steps=500, exists_ok=False)
    
    num_caught_ff = len(rl.ff_caught_T_sorted)
        
    result = {'dv': rl.env_for_data_collection.dv_cost_factor, 
              'jerk': rl.env_for_data_collection.jerk_cost_factor, 
              'stop': rl.env_for_data_collection.cost_per_stop, 
              'num_caught_ff': num_caught_ff,
              'num_stops': (rl.monkey_information['monkey_speeddummy']==0).sum(),
              'agent_name': agent_name,
              }
    cost_results = pd.concat([cost_results, pd.DataFrame([result])], ignore_index=True)
        


## to make animation too

In [ ]:
from IPython.display import HTML
for agent_dir in dirs:
    agent_name = os.path.basename(agent_dir)
    print('==============================================')
    print(agent_name)
    

    
    rl = rppo_class.RPPOforMultifirefly(overall_folder=str(agent_dir))

    rl.streamline_making_animation(currentTrial_for_animation=None, num_trials_for_animation=None, duration=[10, 20], n_steps=250, video_dir=None,
                                  data_exists_ok=True, display_inline=False,
                                  save_video=False)


    num_caught_ff = len(rl.ff_caught_T_sorted)

    # Convert animation to HTML
    html_str = rl.anim.to_jshtml()

    video_path_name = os.path.join(base_path, 'derived_agent_animation', f'{agent_name}_C_{num_caught_ff}.html')
    os.makedirs(os.path.dirname(video_path_name), exist_ok=True)
    with open(video_path_name, 'w') as f:
        f.write(html_str)
        print(f"Animation saved to {video_path_name}")
        
    print(rl.monkey_information['speed'].values[:30])
        

    # from IPython.display import HTML
    # HTML(rl.anim.to_jshtml()) 

In [ ]:
rl.monkey_information['speed'].values[:30]

In [ ]:
# from stable_baselines3.common.evaluation import evaluate_policy

# # Ensure env/agent exist; your animation call likely created them
# # If needed:
# # rl.make_env(**env_kwargs)
# # rl.make_agent()

# avg_ret, std_ret = evaluate_policy(
#     rl.rl_agent,               # RecurrentPPO model
#     rl.env,                    # VecMonitor-wrapped env
#     n_eval_episodes=10,
#     deterministic=True,
# )
# print(f"Avg return: {avg_ret:.2f} ± {std_ret:.2f}")

In [ ]:
# rl.make_env(**rl.env_kwargs)
# rl.make_agent()
# # rl.make_initial_env_for_curriculum_training()
# rl.load_best_model_postcurriculum()
rl.streamline_making_animation(currentTrial_for_animation=None, num_trials_for_animation=None, duration=[10, 12], n_steps=1000, video_dir=None,
                               data_exists_ok=False, display_inline=False,
                               save_video=True)

# Convert animation to HTML
html_str = rl.anim.to_jshtml()
with open(self.video_path_name, 'w') as f:
    f.write(html_str)
    
# from IPython.display import Video
# Video(rl.video_path_name, embed=True)

# from IPython.display import HTML
# HTML(rl.anim.to_jshtml())  # or HTML(your_instance.anim.to_jshtml())

## manually eval

In [ ]:
reload(lstm_utils)

In [ ]:
avg_reward = lstm_utils.evaluate_agent(
    rl.env, rl.rl_agent, 512, 2, deterministic=True)
print(
    f"Best average reward: {rl.best_avg_reward}, Current average reward: {avg_reward}")


# Sweep results

In [ ]:
import pandas as pd
import glob
import os

log_dir = 'multiff_analysis/logs/rppo/run_curriculum/160811'
files = [f for f in glob.glob(os.path.join(log_dir, '*.csv'))]

# log_dir = 'multiff_analysis/logs/rppo/run_summary/160811'
# files = [f for f in glob.glob(os.path.join(log_dir, '*.csv'))]

if len(files) == 0:
    raise ValueError(f"No files found in {log_dir}")

records = []

for f in files:
    df = pd.read_csv(f)
    if df.empty:
        raise ValueError(f"File {f} is empty")

    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.dropna(subset=['timestamp'])
    if df.empty:
        raise ValueError(f"File {f} has no valid timestamps")
        

    df = df.sort_values('stage')
    last_stage = df.iloc[-1]

    train_time = (df['timestamp'].iloc[-1] - df['timestamp'].iloc[0]).total_seconds() / 60  # minutes

    # Extract run ID (last part of overall_folder)
    run_id = os.path.basename(str(last_stage['overall_folder']))

    records.append({
        'file': os.path.basename(f),
        'run_id': run_id,  # ← added here
        'agent_type': last_stage['agent_type'],
        'lr': last_stage['param_lr'],
        'num_obs_ff': last_stage['param_num_obs_ff'],
        'max_in_memory_time': last_stage['param_max_in_memory_time'],
        'strategy': last_stage['param_identity_slot_strategy'],
        'n_envs': last_stage['param_n_envs'],
        'angular_terminal_vel': last_stage['param_angular_terminal_vel'],
        #'final_stage': last_stage['stage'],
        'best_reward_last': last_stage['best_avg_reward'],
        'reward_threshold': last_stage['reward_threshold'],
        'flash_on_interval': last_stage['flash_on_interval'],
        'train_time_min': train_time,
        'finished_curriculum': last_stage['finished_curriculum'],
        'overall_folder': last_stage['overall_folder']
    })

summary = pd.DataFrame(records)
summary = summary.sort_values('best_reward_last', ascending=False)

print(summary[['run_id', 'num_obs_ff', 'strategy', 'best_reward_last', 'train_time_min', 'max_in_memory_time']])


In [ ]:
summary = summary[summary['finished_curriculum'] == True].copy()
# summary.loc[summary['finished_curriculum'] == False, 'train_time_min'] = max(summary['train_time_min'] + 100)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.lines import Line2D

# --- Setup ---
strategies = summary['strategy'].unique()
palette = sns.color_palette('tab10', len(strategies))
color_map = dict(zip(strategies, palette))

plt.figure(figsize=(7, 5))

# --- Marker size scaling ---
min_size, max_size = 50, 300  # scatter point area (points^2)
summary['marker_size'] = np.interp(
    summary['max_in_memory_time'],
    (summary['max_in_memory_time'].min(), summary['max_in_memory_time'].max()),
    (min_size, max_size)
)

# --- Scatter plot ---
for strat in strategies:
    subset = summary[summary['strategy'] == strat]
    plt.scatter(
        subset['train_time_min'],
        subset['best_reward_last'],
        label=strat,
        color=color_map[strat],
        alpha=0.8,
        edgecolor='black',
        s=subset['marker_size']
    )

# --- Text annotations (num_obs_ff) ---
# Label all if few points, or just a representative subset if large
if len(summary) <= 100:
    annotated = summary
else:
    annotated = summary.nlargest(15, 'best_reward_last')  # top 15 for readability

for _, row in annotated.iterrows():
    plt.text(row['train_time_min'] + 0.5,
             row['best_reward_last'] + 3,
             str(row['num_obs_ff']),
             fontsize=10,
             alpha=0.8)

# --- Legends ---
# Color legend (strategy)
color_handles = [
    Line2D([0], [0], marker='o', color='w',
           markerfacecolor=color_map[strat], markeredgecolor='black',
           label=strat, markersize=10)
    for strat in strategies
]

# Size legend (max_in_memory_time)
size_values = np.linspace(summary['max_in_memory_time'].min(),
                          summary['max_in_memory_time'].max(), 3)
size_handles = [
    Line2D([0], [0], marker='o', color='w',
           markerfacecolor='gray', markeredgecolor='black',
           label=f'{val:.1f} s',
           markersize=np.sqrt(np.interp(val,
                                        (summary['max_in_memory_time'].min(),
                                         summary['max_in_memory_time'].max()),
                                        (min_size, max_size))) / 2)
    for val in size_values
]

first_legend = plt.legend(handles=color_handles, title='Strategy',
                          loc='upper left', bbox_to_anchor=(1.02, 1))
second_legend = plt.legend(handles=size_handles, title='Max In-Memory Time',
                           loc='lower left', bbox_to_anchor=(1.02, 0))
plt.gca().add_artist(first_legend)

# --- Aesthetics ---
plt.xlabel('Training Time (minutes)')
plt.ylabel('Best Avg Reward (Last Stage)')
plt.title('Reward vs. Training Time by Strategy')
plt.tight_layout()
plt.gca().invert_xaxis()
plt.show()


In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 5))
sns.barplot(data=summary, x='strategy', y='best_reward_last', hue='num_obs_ff', palette='muted')
plt.ylabel('Best Avg Reward')
plt.title('Final Performance by Strategy and num_obs_ff')
plt.legend(title='num_obs_ff')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 5))
sns.barplot(
    data=summary,
    x='strategy',
    y='train_time_min',
    hue='num_obs_ff',
    palette='muted'
)
plt.ylabel('Training Time (minutes)')
plt.xlabel('Identity Slot Strategy')
plt.title('Total Training Time by Strategy and num_obs_ff')
plt.legend(title='num_obs_ff')
plt.tight_layout()
plt.show()


In [ ]:
stop!

# Optuna (LSTM)

(my own codes)

##### sample_sac_params

In [ ]:
def sample_sac_params(trial):
    """
    Sampler for SAC hyperparams.
    :param trial: (optuna.trial)
    :return: (dict)
    """
  
    gamma = 1.0 - trial.suggest_float("1-gamma", 1e-4, 0.1, log=True)
    soft_q_lr = trial.suggest_float("soft_q_lr", 1e-5, 1, log=True)
    policy_lr = trial.suggest_float("policy_lr", 1e-5, 1, log=True)
    alpha_lr  = trial.suggest_float("alpha_lr", 1e-5, 1, log=True)
    batch_size  = trial.suggest_categorical('batch_size', [5, 10, 15, 20, 25, 30])
    update_itr = trial.suggest_categorical('update_itr', [1, 2, 3, 5])
    hidden_dim = trial.suggest_categorical('hidden_dim', [16, 32, 64, 100, 150, 200, 256])
    reward_scale = trial.suggest_categorical('reward_scale', [1, 3, 5, 10, 15, 20]) # I updated after running
    target_entropy = trial.suggest_categorical('target_entropy', [-1, -2, -3, -5, -8, -10]) # I updated after running
    soft_tau= trial.suggest_float("soft_tau", 1e-6, 1, log=True)
    #activation_fn


    return {
        'gamma': gamma,
        'soft_q_lr':soft_q_lr,
        'policy_lr':policy_lr,
        'alpha_lr':alpha_lr,
        'batch_size': batch_size,
        'update_itr':update_itr,
        'hidden_dim': hidden_dim,
        'reward_scale':reward_scale,
        'target_entropy':target_entropy,
        'soft_tau':soft_tau
    }

## put_in_fixed_params

In [ ]:
def put_in_fixed_params():
    return {
        'model_folder_name':  None, 
        'train_freq': 100, 
        'batch_size': 10, 
        'update_itr': 1,
        'num_train_episodes': 50, 
        'eval_eps_freq': 10, 
        'max_steps_per_eps': 1024, 
        'auto_entropy': True, 
        'DETERMINISTIC': False, 
        'num_eval_episodes': 3, 
        'print_episode_reward':  True}

## objective

In [ ]:
def objective(trial: optuna.Trial) -> float: 
  try:
    # Sample hyperparameters
    kwargs = sample_sac_params(trial)
    kwargs = put_in_fixed_params()

    num_train_episodes = kwargs['num_train_episodes'] 
    eval_eps_freq = kwargs['eval_eps_freq']
    max_steps_per_eps = kwargs['max_steps_per_eps'] 
    auto_entropy = kwargs['auto_entropy'] 
    num_eval_episodes = kwargs['num_eval_episodes'] 
    print_episode_reward = kwargs['print_episode_reward']


    env = rnn_env.EnvForRNN()
    rl_agent = lstm_utils.SAC_Trainer(**kwargs)
    rl2 = lstm_class.LSTMforMultifirefly()
    rl2.rl_agent = rl_agent
    rl_agent, best_avg_reward_record, alpha_df = rl2.train_agent(env, device,
                                                         num_train_episodes=num_train_episodes, eval_eps_freq=eval_eps_freq, max_steps_per_eps=max_steps_per_eps, 
                                                         num_eval_episodes=num_eval_episodes, print_episode_reward=print_episode_reward, auto_entropy=auto_entropy)

  except ValueError as e:
    # Sometimes, random hyperparams can generate NaN
      print(e)

  return best_avg_reward_record

## run

In [ ]:
N_TRIALS = 100
N_STARTUP_TRIALS = 5

# Set pytorch num threads to 1 for faster training
torch.set_num_threads(1)
 
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
## Do not prune before 1/3 of the max budget is used
# pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS 2//3 3)

study = optuna.create_study(sampler=sampler, direction="maximize")
try:
    study.optimize(objective, n_trials=N_TRIALS)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print("    {}: {}".format(key, value))